In [1]:
import numpy as np
import pandas as pd
import cv2 
from PIL import Image
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

In [2]:
y = np.load('new_y.npy')
x = np.load('new_cut_feature.npy')
x.shape , y.shape

((1900, 138), (1900,))

In [3]:
x_ar = []
x_en = []

y_ar = []
y_en = []
for i,v in enumerate(x):
    if i%4<2:
        x_ar.append(v)
        y_ar.append(y[i])
    else:
        x_en.append(v)
        y_en.append(y[i])
        
x_ar , y_ar , x_en , y_en= np.array(x_ar),np.array(y_ar),np.array(x_en),np.array(y_en)

print(x_ar.shape , x_en.shape , y_ar.shape , y_en.shape)

(950, 138) (950, 138) (950,) (950,)


In [30]:
#train test = 7 : 3

#아랍 데이터 나누기
x_train_ar, x_test_ar, y_train_ar, y_test_ar = train_test_split(x_ar,y_ar, test_size=0.3, shuffle=True, random_state=111)

#영어 데이터 나누기
x_train_en, x_test_en, y_train_en, y_test_en = train_test_split(x_en,y_en, test_size=0.3, shuffle=True, random_state=111)

#영아 + 아랍 데이터
x_train_all = np.concatenate((x_train_ar,x_train_en),axis=0)
x_test_all =  np.concatenate((x_test_ar,x_test_en),axis=0)

y_train_all = np.concatenate((y_train_ar,y_train_en),axis=0)
y_test_all = np.concatenate((y_test_ar,y_test_en),axis=0)

print(x_train_ar.shape , x_test_ar.shape)
print(y_train_ar.shape , y_test_ar.shape)

print(x_train_en.shape , x_test_en.shape)
print(y_train_en.shape , y_test_en.shape)
      
print(x_train_all.shape , x_test_all.shape)
print(y_train_all.shape , y_test_all.shape)

(665, 138) (285, 138)
(665,) (285,)
(665, 138) (285, 138)
(665,) (285,)
(1330, 138) (570, 138)
(1330,) (570,)


In [36]:
#Case마다 여기서 변경
train_x = x_train_en
test_x = x_test_all

train_y = y_train_en
test_y = y_test_all

cs = StandardScaler()
train_x = cs.fit_transform(train_x)
test_x = cs.transform(test_x)



#모두 False나와야 함
print(np.isnan(train_x).all(),np.isnan(test_x).all(),
      np.isnan(train_y).all(),np.isnan(test_y).all())
#모두 True 나와야 함
print(np.isfinite(train_x).all(),np.isfinite(test_x).all(),
      np.isfinite(train_y).all(),np.isfinite(test_y).all())

print(f'train data length: {len(train_x)} , {len(train_y)}')
print(f'train data length: {len(test_x)} , {len(test_y)}')

False False False False
True True True True
train data length: 665 , 665
train data length: 570 , 570


SVM

In [6]:
pd.set_option('display.max.colwidth', 100)

In [7]:
param_grid = {'kernel':['rbf'], 'C':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
              'gamma':['auto','scale',0.0001,0.0005,0.001,0.002,0,0.005,0.007,0.01,0.02,0.05]}

svm_clf = SVC(random_state=100)
grid = GridSearchCV(svm_clf, param_grid=param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
grid.fit(train_x, train_y)

print(grid.best_score_,grid.best_params_)
print(pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score')[['rank_test_score','params','mean_test_score']][:5])

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   21.4s finished


0.780451127819549 {'C': 3, 'gamma': 0.02, 'kernel': 'rbf'}
     rank_test_score                                     params  \
34                 1   {'C': 3, 'gamma': 0.02, 'kernel': 'rbf'}   
81                 2   {'C': 7, 'gamma': 0.01, 'kernel': 'rbf'}   
153                3  {'C': 13, 'gamma': 0.01, 'kernel': 'rbf'}   
57                 3   {'C': 5, 'gamma': 0.01, 'kernel': 'rbf'}   
69                 5   {'C': 6, 'gamma': 0.01, 'kernel': 'rbf'}   

     mean_test_score  
34          0.780451  
81          0.779699  
153         0.778947  
57          0.778947  
69          0.778195  


In [26]:
svm_pred = grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, svm_pred)*100, '%')

Accuracy: 77.89473684210526 %


rf

In [15]:
rf_param_grid = {
    'n_estimators' : [270,280,290,300],
    'max_depth' : [9,10,11,12,13],      #default : " None",
    'min_samples_leaf' : [1,2],     #default : 1
    'min_samples_split' : [2,3,4,5]  #default : 2
}

rf_clf = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(rf_clf, param_grid=rf_param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
rf_grid.fit(train_x, train_y)

print(rf_grid.best_score_,rf_grid.best_params_)
rf_df = pd.DataFrame(rf_grid.cv_results_).sort_values(by='rank_test_score')
print(rf_df[['params','mean_test_score']][:5])

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  2.0min finished


0.7300751879699249 {'max_depth': 12, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 280}
                                                                                    params  \
101  {'max_depth': 12, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 280}   
100  {'max_depth': 12, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 270}   
137  {'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 280}   
132  {'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 270}   
139  {'max_depth': 13, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 300}   

     mean_test_score  
101         0.730075  
100         0.728571  
137         0.727820  
132         0.726316  
139         0.725564  


In [27]:
rf_pred = rf_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, rf_pred)*100, '%')

Accuracy: 69.12280701754386 %


XGB

In [17]:
xgb_param={
    'n_estimators' : [300,310],
    'learning_rate' : [0.15,0.2,0.25],
    'max_depth' : [5,6,7,8,9],
    #'gamma' : [0],
    'colsample_bytree' : [0.9],
    
}
xgb_clf= xgb.XGBClassifier(random_state=42)#tree_method='gpu_hist', gpu_id=0,
xgb_grid = GridSearchCV(xgb_clf, param_grid=xgb_param,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
xgb_grid.fit(train_x, train_y)

print(xgb_grid.best_score_,xgb_grid.best_params_)
xgb_df = pd.DataFrame(xgb_grid.cv_results_).sort_values(by='rank_test_score')
print(xgb_df[['params','mean_test_score']][:5])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.0min finished


0.7541353383458647 {'colsample_bytree': 0.9, 'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 300}
                                                                                   params  \
12   {'colsample_bytree': 0.9, 'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 300}   
13   {'colsample_bytree': 0.9, 'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 310}   
28  {'colsample_bytree': 0.9, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 300}   
29  {'colsample_bytree': 0.9, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 310}   
17   {'colsample_bytree': 0.9, 'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 310}   

    mean_test_score  
12         0.754135  
13         0.752632  
28         0.752632  
29         0.752632  
17         0.751880  


In [28]:
xgb_pred = xgb_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, xgb_pred)*100, '%')

Accuracy: 72.98245614035088 %


Logistic regression

In [32]:
param_grid = {'penalty':['l1','l2'], 
              'C':[0.001,0.005,0.01,0.05,0.1,0.3,0.5], 
              'max_iter':[100,150,200,250,300,500]}

log_clf = LogisticRegression(random_state = 42)
log_grid = GridSearchCV(log_clf, param_grid=param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
log_grid.fit(train_x, train_y)
print(log_grid.best_score_,log_grid.best_params_)
print(pd.DataFrame(log_grid.cv_results_).sort_values(by='rank_test_score')[['rank_test_score','params','mean_test_score']][:5])

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    5.3s


0.7488721804511278 {'C': 0.05, 'max_iter': 100, 'penalty': 'l2'}
    rank_test_score                                         params  \
41                1  {'C': 0.05, 'max_iter': 200, 'penalty': 'l2'}   
37                1  {'C': 0.05, 'max_iter': 100, 'penalty': 'l2'}   
39                1  {'C': 0.05, 'max_iter': 150, 'penalty': 'l2'}   
43                1  {'C': 0.05, 'max_iter': 250, 'penalty': 'l2'}   
45                1  {'C': 0.05, 'max_iter': 300, 'penalty': 'l2'}   

    mean_test_score  
41         0.748872  
37         0.748872  
39         0.748872  
43         0.748872  
45         0.748872  


[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:    6.2s finished


In [37]:
log_pred = log_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, log_pred)*100, '%')

Accuracy: 70.52631578947368 %
